## Import

In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [3]:
train = pd.read_csv('./open/train.csv')

## Model Define, Train and Inference

In [4]:
print(train)

              일자     종목코드     종목명      거래량     시가     고가     저가     종가
0       20210601  A060310      3S   166690   2890   2970   2885   2920
1       20210601  A095570  AJ네트웍스    63836   5860   5940   5750   5780
2       20210601  A006840   AK홀딩스   103691  35500  35600  34150  34400
3       20210601  A054620     APS   462544  14600  14950  13800  14950
4       20210601  A265520   AP시스템   131987  29150  29150  28800  29050
...          ...      ...     ...      ...    ...    ...    ...    ...
987995  20230530  A189980  흥국에프엔비   272284   3005   3035   2955   2980
987996  20230530  A000540    흥국화재    50218   3250   3255   3195   3215
987997  20230530  A003280    흥아해운   130664   1344   1395   1340   1370
987998  20230530  A037440      희림   141932   9170   9260   9170   9200
987999  20230530  A238490      힘스  2611843   6410   8220   6300   8220

[988000 rows x 8 columns]


In [20]:
## UDF for ADF test
from statsmodels.tsa.stattools import adfuller
import pandas as pd

def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [23]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code][['일자', '거래량', '시가', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    tc = train_close['종가'] - train_close['시가']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = sum(predictions.iloc)
    
    # 결과 저장
    results_df = pd.concat([results_df, pd.DataFrame({'종목코드': [code], 'final_return': [final_return]})], ignore_index=True)

100%|██████████| 2000/2000 [07:17<00:00,  4.57it/s]


In [25]:
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,final_return,순위
0,A060310,39.781207,448
1,A095570,37.326538,456
2,A006840,4.534800,577
3,A054620,22.397960,497
4,A265520,-615.045281,1708
...,...,...,...
1995,A189980,-1.350804,614
1996,A000540,20.494919,506
1997,A003280,102.428380,346
1998,A037440,-28.268502,765


## Submit

In [26]:
sample_submission = pd.read_csv('./open/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [27]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,1516
1,A000040,859
2,A000050,1434
3,A000070,1936
4,A000080,1688
...,...,...
1995,A375500,1858
1996,A378850,993
1997,A383220,16
1998,A383310,1717


In [28]:
baseline_submission.to_csv('ARIMA_2_1_2_return_submission.csv', index=False)